# Vorbereitungen

In [ ]:
!pip install openai pydantic langchain langchain-openai

In [ ]:
# OPENAI KEY lesen
import os
try:
    from google.colab import userdata
    OPENAI_KEY = userdata.get('OPENAI_KEY')
except:
    OPENAI_KEY = os.getenv('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_KEY


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

## Simpler LLM Aufruf

In [ ]:
llm.invoke("Was ist Langchain?")

### SystemMessage, HumanMessage, AIMessage

In [ ]:
system = SystemMessage('Du bist ein lustiger Typ dem zu jeder Nachricht ein Witz einfällt.')
human = HumanMessage('Kennst du ein Donauwellenrezept?')

In [ ]:
messages = [system, human]

In [ ]:
response = llm.invoke(messages)
response

In [ ]:
messages.append(response)

In [ ]:
human2 = HumanMessage('Lustig, aber kennst du wirklich ein Rezept?')
messages.append(human2)

In [ ]:
messages

In [ ]:
llm.invoke(messages)

## Prompts mit Langchain + LCEL (Langchain Expression Language)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Du bist ein lustiger Typ dem zu jeder Nachricht ein Witz einfällt."),
    ("user", "{input}")
])

In [ ]:
chain = prompt | llm 

In [ ]:
chain.invoke({"input": "Was ist Langchain?"})

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
chain = prompt | llm | StrOutputParser()

In [ ]:
chain.invoke({"input": "Was ist Langchain?"})

### Prompt Chaining

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
prompt1 = ChatPromptTemplate.from_template(
    "Generiere eine {attribute} Farbe. Gebe nur den Namen der Farbe zurück - keinen weiteren Text:"
)
prompt2 = ChatPromptTemplate.from_template(
    "Welche Frucht hat die Farbe: {color}. Gebe den Namen der Frucht aus und schreibe ein Gedicht über sie!"
)

In [ ]:
chain = prompt1 | llm 

In [ ]:
chain.invoke({'attribute': 'fröhlich'})

Der `prompt2` erwartet `"color"` als Input - keine "AIMessage".

In [ ]:
chain = prompt1 | llm | StrOutputParser()
chain.invoke({'attribute': 'fröhlich'})

In [ ]:
chain = prompt1 | llm | StrOutputParser() | {"color": RunnablePassthrough()}
chain.invoke({'attribute': 'fröhlich'})

Wir können einen Teil der Chain vereinfachen - `llm + StrOutputParser()` fassen wir zusammen - das macht dann den `RunnablePassthrough()` überflüssig!

In [ ]:
model_parser = llm | StrOutputParser()
chain = prompt1 | {"color": model_parser}
chain.invoke({'attribute': 'fröhlich'})

In [ ]:
model_parser = llm | StrOutputParser()
chain = prompt1 | {"color": model_parser} | prompt2 | model_parser
chain.invoke({'attribute': 'fröhlich'})